# Calculate Urban Metrics: Fullness
Fraction of the total area of the urban extent that is built-up. Use a custom script that takes the mean value of all the pixels of the GHS built-up within the urban extent. The GHS built-up layer values are expressed as decimals from 0 to 1, and correspond to the fraction of the pixel that is covered by a building (Pesaresi et al., 2016)

In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import sys, os, inspect, logging, importlib, time

In [57]:
import pandas as pd
import geopandas as gpd
import numpy as np
import math, random

In [58]:
import shapely
from shapely.geometry import mapping
from shapely.geometry import Point

In [59]:
# Get reference to GOSTNets
sys.path.append(r'C:\repos\INFRA_SAP')
from infrasap.urban_metrics import *

In [60]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine # or from affine import Affine

In [61]:
import time
start_time = time.time()

## Inputs are GHS built and the urban extents
Mollweide projection should work good because it is an equal-area projection

In [62]:
built_up_layer = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\GHS_BUILT_LDS2014_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS2014_GLOBE_R2018A_54009_250_V2_0.tif"
#built_up_layer = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\2015_250m_GHS_Built\UZB_2015_GHS_built_merged.tif"

In [63]:
#shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\russia_urban_extents_merged_mollweide.shp"
#shpName = r"C:\repos\GOST_Urban\Notebooks\Implementations\eca_wo_rus_urban_clusters_ghs_pop_smooth_100k_mollweide2.shp"
#shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_FUAs_Project_Mollweide.shp"
#shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_only_ghs_built_up_extents_mollweide_geom_fixed_greater_50k.shp"
#shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_ghs_built_up_extents_mollweide_geom_fixed.shp"
#shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\ECA_wo_rus_urban_extents\eca_wo_rus_built_up_extents_molleweide.shp"
shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_clusters_5k_up_molleweide.shp"

In [64]:
input_shapes_gpd = gpd.read_file(shpName)
input_shapes_gpd.head(5)

,ID,Pop,layer,path,geometry
0,2038,5213.978912,None,None,"POLYGON ((2007000.000 6478000.000, 2010000.000..."
1,2355,8086.003265,None,None,"POLYGON ((1952000.000 6468000.000, 1953000.000..."
2,2691,12046.157654,None,None,"POLYGON ((1893000.000 6452000.000, 1897000.000..."
3,2753,11929.881653,None,None,"POLYGON ((1934000.000 6450000.000, 1937000.000..."
4,3060,115285.056396,None,None,"POLYGON ((2010000.000 6444000.000, 2011000.000..."


In [65]:
%%time

with rasterio.open(built_up_layer) as src:
    pixelSizeX, pixelSizeY  = src.res
    #print(pixelSizeX, pixelSizeY)
    pixel_area = 250**2
    
    input_shapes_gpd = gpd.read_file(shpName)
    
    # psuedocode
    # For each Shape:
        # Select all built-up pixels that are mostly within shape
        # Area of shape = sum of all pixels * area of each pixel
        # Built-up Area = (sum pixel value of each pixel) / 100
        # Fullness index = Built-up Area / Area of Shape

    #for entry in input_shapes_gpd.head(5).iterrows():
    for entry in input_shapes_gpd.iterrows():
        print(entry[0])
        #print(row[1]['geometry'])

        # extract the geometry in GeoJSON format
        geometry = entry[1]['geometry'] # list of shapely geometries
        #geometry = geoms[0] # shapely geometry
        geoms = [mapping(geometry)]

        # extract the raster values values within the polygon 
        out_image, out_transform = mask(src, geoms, crop=True, nodata=-9999.0)
        data = out_image[0,:,:]
        
        row, col = np.where(data != -9999.0) 
        val = np.extract(data != -9999.0, data)

        T1 = out_transform * Affine.translation(0.5, 0.5) # reference the pixel centre
        rc2xy = lambda r, c: (c, r) * T1  
        
        d = gpd.GeoDataFrame({'col':col,'row':row,'val':val})
        
        # coordinate transformation
        d['x'] = d.apply(lambda row: rc2xy(row.row,row.col)[0], axis=1)
        d['y'] = d.apply(lambda row: rc2xy(row.row,row.col)[1], axis=1)
        
        # geometry
        d['geometry'] = d.apply(lambda row: Point(row['x'], row['y']), axis=1)
        
        # Area of shape = sum of all pixels * area of each pixel
        area_of_shape = pixel_area * d.count()[0]
        # Built-up Area = (sum pixel value of each pixel) / 100
        d2 = d.val/100
        built_up = sum(d2) * pixel_area
        # Fullness index = Built-up Area / Area of Shape
        fullness_index = built_up / area_of_shape
        print(f"fullness index: {fullness_index}")
        
        # creates a temporary GDF for just the row's shape
        temp_gdf = input_shapes_gpd.iloc[[entry[0]]]
        
        #print("print temp_gdf")
        #print(temp_gdf)
        
        # Put all metrics in a DataFrame
        metrics_scalar = {}
        metrics_scalar['fullness_index'] = [fullness_index]
        metrics_df = pd.DataFrame(metrics_scalar)
        
        #print("print metrics_scalar")
        #print(metrics_scalar)
        
        # and concatinate it with the row's shape
        new_temp_gdf = pd.concat([temp_gdf.reset_index(drop=True), metrics_df], axis=1)
        
        #print("print new_temp_gdf")
        #print(new_temp_gdf)
        #print(entry[0])
        # put the results of each row into a new DataFrame
        if entry[0] == 0:
            print("new_temp_gdf")
            output_new_temp_gdf = new_temp_gdf
        else:
            output_new_temp_gdf = output_new_temp_gdf.append(new_temp_gdf, ignore_index=True)       

0
fullness index: 0.17849416493496997
new_temp_gdf
1
fullness index: 0.15169333337366375
2
fullness index: 0.26024469751780543
3
fullness index: 0.24227137479829253
4
fullness index: 0.2781471610462591
5
fullness index: 0.3657316739037044
6


C:\Users\war-machine\Anaconda2\envs\gostnets2\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Right multiplication will be prohibited in version 3.0


fullness index: 0.23789181697859682
7
fullness index: 0.2665093317530894
8
fullness index: 0.31645799876241654
9
fullness index: 0.21858828689427615
10
fullness index: 0.5073773000637382
11
fullness index: 0.3903360011573265
12
fullness index: 0.3339517636032885
13
fullness index: 0.22884044584629412
14
fullness index: 0.3755587159913765
15
fullness index: 0.3372259994325254
16
fullness index: 0.30649914403537487
17
fullness index: 0.20784891028729777
18
fullness index: 0.36075349977988935
19
fullness index: 0.49451823225079855
20
fullness index: 0.23798059967994048
21
fullness index: 0.29805818185820954
22
fullness index: 0.28410856353559666
23
fullness index: 0.36903916507435497
24
fullness index: 0.21882390925152737
25
fullness index: 0.1852290008383635
26
fullness index: 0.40715816730511284
27
fullness index: 0.3407468337988272
28
fullness index: 0.29425800012541004
29
fullness index: 0.37309037709837867
30
fullness index: 0.27452020105120023
31
fullness index: 0.49338556521578075


216
fullness index: 0.5151125870952186
217
fullness index: 0.6373367349626236
218
fullness index: 0.31131840187917986
219
fullness index: 0.2260314441882656
220
fullness index: 0.2312437139670395
221
fullness index: 0.5516003200159372
222
fullness index: 0.3745393176811633
223
fullness index: 0.533983581938628
224
fullness index: 0.33166466597928473
225
fullness index: 0.4099997493724004
226
fullness index: 0.24957175161489434
227
fullness index: 0.23162358321981932
228
fullness index: 0.2592356237518345
229
fullness index: 0.29275770918437394
230
fullness index: 0.40578043846280687
231
fullness index: 0.3379305712410964
232
fullness index: 0.2531796996427147
233
fullness index: 0.6158911217680725
234
fullness index: 0.23290775026058932
235
fullness index: 0.3238547643455497
236
fullness index: 0.3690884994530279
237
fullness index: 0.20477166775162914
238
fullness index: 0.2793776921716926
239
fullness index: 0.26652159990947744
240
fullness index: 0.22789500027660584
241
fullness ind

429
fullness index: 0.33844214436560704
430
fullness index: 0.2515914284372879
431
fullness index: 0.32038866908139124
432
fullness index: 0.16825428649618906
433
fullness index: 0.3888229183645996
434
fullness index: 0.1066545000289807
435
fullness index: 0.3521937266321567
436
fullness index: 0.2819171425501281
437
fullness index: 0.30211000155083767
438
fullness index: 0.23491610663367474
439
fullness index: 0.4835213013104294
440
fullness index: 0.35723287577837937
441
fullness index: 0.26378677717305743
442
fullness index: 0.29222272166488333
443
fullness index: 0.3781446004464669
444
fullness index: 0.07806819963007001
445
fullness index: 0.15074566689994148
446
fullness index: 0.35103114267682095
447
fullness index: 0.2383732318957947
448
fullness index: 0.1891574296708508
449
fullness index: 0.47003659311279833
450
fullness index: 0.19074574991554982
451
fullness index: 0.41007419887318974
452
fullness index: 0.32855004306196384
453
fullness index: 0.4269074486125239
454
fullne

...

7012
fullness index: 0.23968733256373928
7013
fullness index: 0.2695035995911894
7014
fullness index: 0.27960457102848785
7015
fullness index: 0.12059737521786928
7016
fullness index: 0.21978371338951502
7017
fullness index: 0.10014742818644404
7018
fullness index: 0.3583118345043962
7019
fullness index: 0.11316357221065639
7020
fullness index: 0.3812650632783061
7021
fullness index: 0.16801722209085154
7022
fullness index: 0.3057950706323546
7023
fullness index: 0.3190836201103507
7024
fullness index: 0.3431471426702046
7025
fullness index: 0.27599772701255043
7026
fullness index: 0.24068699838244356
7027
fullness index: 0.2585695017890733
7028
fullness index: 0.4254137729416062
7029
fullness index: 0.524554060359819
7030
fullness index: 0.2546778032256982
7031
fullness index: 0.418893750128215
7032
fullness index: 0.3788492784531425
7033
fullness index: 0.3494843592969119
7034
fullness index: 0.509382200113032
7035
fullness index: 0.28548949986344496
7036
fullness index: 0.4502146967

fullness index: 0.24507760060555484
7222
fullness index: 0.20660204671176394
7223
fullness index: 0.31542962619096215
7224
fullness index: 0.26551490786567394
7225
fullness index: 0.3237942515988834
7226
fullness index: 0.323976076841623
7227
fullness index: 0.19230540050339187
7228
fullness index: 0.4584853743439302
7229
fullness index: 0.3456795553329736
7230
fullness index: 0.44344071007799357
7231
fullness index: 0.2626552490677341
7232
fullness index: 0.3212086998639279
7233
fullness index: 0.19330391634275657
7234
fullness index: 0.2137772843896073
7235
fullness index: 0.29367872728140687
7236
fullness index: 0.44665062388619015
7237
fullness index: 0.36851314702955645
7238
fullness index: 0.47156929842767564
7239
fullness index: 0.5266432500348401
7240
fullness index: 0.4946379416993734
7241
fullness index: 0.27992233351720747
7242
fullness index: 0.37010085641772356
7243
fullness index: 0.2701230896537626
7244
fullness index: 0.34547350436253926
7245
fullness index: 0.335569233

7424
fullness index: 0.3477766672141216
7425
fullness index: 0.3836746090100464
7426
fullness index: 0.47375561695569707
7427
fullness index: 0.20781657085152413
7428
fullness index: 0.4175305243323324
7429
fullness index: 0.3785436661661758
7430
fullness index: 0.4461284432409861
7431
fullness index: 0.36750700027675975
7432
fullness index: 0.3486341432373939
7433
fullness index: 0.08933216604539969
7434
fullness index: 0.11508818516199004
7435
fullness index: 0.4850217487801274
7436
fullness index: 0.38426477041405005
7437
fullness index: 0.1449334011575047
7438
fullness index: 0.12699149909129423
7439
fullness index: 0.39460944703541323
7440
fullness index: 0.3854412016662536
7441
fullness index: 0.16742812361508186
7442
fullness index: 0.46562845714467715
7443
fullness index: 0.1648340992631347
7444
fullness index: 0.5687695709634032
7445
fullness index: 0.24475066785251126
7446
fullness index: 0.2752004991749004
7447
fullness index: 0.289833299685381
7448
fullness index: 0.4349808

7632
fullness index: 0.43982885938126853
7633
fullness index: 0.39545888941478957
7634
fullness index: 0.4114201991327718
7635
fullness index: 0.27907757083136986
7636
fullness index: 0.3324967685599894
7637
fullness index: 0.488861147600683
7638
fullness index: 0.36917066660721
7639
fullness index: 0.3377823342064706
7640
fullness index: 0.3930575004154946
7641
fullness index: 0.35281888124482336
7642
fullness index: 0.2908574089743493
7643
fullness index: 0.3906499995035423
7644
fullness index: 0.20306260172656038
7645
fullness index: 0.4559947523545513
7646
fullness index: 0.1839750695033737
7647
fullness index: 0.24328535923094022
7648
fullness index: 0.13937166831461428
7649
fullness index: 0.3593215054594718
7650
fullness index: 0.38039928607518697
7651
fullness index: 0.2152781114519813
7652
fullness index: 0.4269162494015291
7653
fullness index: 0.3798759570503455
7654
fullness index: 0.32819270326326294
7655
fullness index: 0.2404428465592201
7656
fullness index: 0.29597683389

7842
fullness index: 0.3592010011987654
7843
fullness index: 0.2536596646750695
7844
fullness index: 0.4110087238444696
7845
fullness index: 0.35698155535404413
7846
fullness index: 0.3731225001147456
7847
fullness index: 0.44824836885814684
7848
fullness index: 0.3988291657636485
7849
fullness index: 0.3570946641775663
7850
fullness index: 0.4927561375096711
7851
fullness index: 0.4464278648100279
7852
fullness index: 0.33700818095770624
7853
fullness index: 0.3505455442725427
7854
fullness index: 0.5327517516308944
7855
fullness index: 0.18091135701175584
7856
fullness index: 0.17699166696790294
7857
fullness index: 0.4982848967008305
7858
fullness index: 0.14374911103065338
7859
fullness index: 0.16231958241390507
7860
fullness index: 0.3202761233005731
7861
fullness index: 0.2234119984638407
7862
fullness index: 0.5546171759505181
7863
fullness index: 0.22831362496964402
7864
fullness index: 0.5420560611762577
7865
fullness index: 0.4939938121483464
7866
fullness index: 0.391461122

In [66]:
output_new_temp_gdf.crs

<Projected CRS: ESRI:54009>
Name: World_Mollweide
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Mollweide
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [67]:
# make the GeoDataFrame unprojected
output_new_temp_gdf = output_new_temp_gdf.to_crs('epsg:4326')

In [68]:
output_new_temp_gdf

,ID,Pop,layer,path,geometry,fullness_index
0,2038,5213.978912,None,None,"POLYGON ((28.77793 55.91414, 28.82094 55.91414...",0.178494
1,2355,8086.003265,None,None,"POLYGON ((27.94342 55.81376, 27.95774 55.81376...",0.151693
2,2691,12046.157654,None,None,"POLYGON ((27.02823 55.65336, 27.08535 55.65336...",0.260245
3,2753,11929.881653,None,None,"POLYGON ((27.60467 55.63333, 27.64749 55.63333...",0.242271
4,3060,115285.056396,None,None,"POLYGON ((28.66157 55.57325, 28.67583 55.57325...",0.278147
...,...,...,...,...,...,...
7993,32,6807.330750,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((105.12725 54.81515, 105.18361 54.815...",0.181821
7994,28,20279.139618,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((109.32656 55.65336, 109.36940 55.653...",0.198175
7995,18,8344.212311,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((107.47868 56.19569, 107.49308 56.195...",0.189358
7996,15,5386.852844,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((116.09091 56.33675, 116.10535 56.336...",0.036448


In [69]:
#output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_metrics_results_russia"
#output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus"
#output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_FUAs_results"
#output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results"
#output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results_all"
#output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus_all"
output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_extents_results_5k_up"

In [70]:
# save as CSV
#output_new_temp_gdf.to_csv(output + r"\ECA_russia_urban_metrics_100k_fullness.csv")
#output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_FUAs_fullness.csv")
#output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_fullness.csv")
#output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_all_fullness.csv")
#output_new_temp_gdf.to_csv(output + r"\ECA_wo_rus_urban_metrics_urban_extents_all_fullness.csv")
output_new_temp_gdf.to_csv(output + r"\all_urban_metrics_5k_up_fullness.csv")

In [71]:
print(f"total time to process: {time.time()-start_time}")

total time to process: 274.2080020904541
